## 导入库

In [3]:
import tensorflow as tf
tfe = tf.contrib.eager

tf.enable_eager_execution()

In [4]:
#在tf.keras.layers包中，图层是对象。 要构建一个图层，
#简单地构造对象。 大多数层将数字作为第一个参数输出尺寸/通道数。
layer = tf.keras.layers.Dense(100)
#输入维度的数量通常是不必要的，因为可以推断
#第一次使用图层，但如果您愿意，可以提供
#specify它手动，这在一些复杂的模型中很有用。
layer = tf.keras.layers.Dense(10,input_shape=(None,5))

In [8]:
# To use a layer, simply call it.
layer(tf.zeros([10,5]))

<tf.Tensor: id=47, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [9]:
#Layers有许多有用的方法。 例如，您可以在一个图层中通过调用layer.variables检查所有变量
#在这种情况下，是完全连接的层将包含权重和偏差的变量。
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.20133352, -0.46263695, -0.03468573,  0.2158491 ,  0.3595522 ,
         -0.58540237,  0.46552366, -0.13607395,  0.15916264,  0.01360285],
        [-0.50026715, -0.2654406 , -0.62901133, -0.43497935, -0.6179841 ,
         -0.21904597, -0.45217654,  0.5424474 ,  0.5907201 , -0.17354006],
        [ 0.24392933,  0.23977798,  0.5685825 ,  0.15353185,  0.4010349 ,
          0.48414987,  0.29462886,  0.6174838 , -0.6137485 ,  0.39350647],
        [ 0.09923446,  0.28053856, -0.47329864,  0.3878029 ,  0.47901493,
         -0.13549611, -0.28698432, -0.62876767, -0.24145219, -0.32751012],
        [ 0.6098836 ,  0.43149465, -0.4627541 ,  0.24477935,  0.28569716,
          0.14579242, -0.22435391,  0.14974087, -0.30755544, -0.11075026]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [10]:
#还可以直接获取所需变量
layer.kernel,layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.20133352, -0.46263695, -0.03468573,  0.2158491 ,  0.3595522 ,
         -0.58540237,  0.46552366, -0.13607395,  0.15916264,  0.01360285],
        [-0.50026715, -0.2654406 , -0.62901133, -0.43497935, -0.6179841 ,
         -0.21904597, -0.45217654,  0.5424474 ,  0.5907201 , -0.17354006],
        [ 0.24392933,  0.23977798,  0.5685825 ,  0.15353185,  0.4010349 ,
          0.48414987,  0.29462886,  0.6174838 , -0.6137485 ,  0.39350647],
        [ 0.09923446,  0.28053856, -0.47329864,  0.3878029 ,  0.47901493,
         -0.13549611, -0.28698432, -0.62876767, -0.24145219, -0.32751012],
        [ 0.6098836 ,  0.43149465, -0.4627541 ,  0.24477935,  0.28569716,
          0.14579242, -0.22435391,  0.14974087, -0.30755544, -0.11075026]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

## 实现自定义图层
实现自己的层的最佳方法是扩展tf.keras.Layer类并实现：

- __init__，您可以在其中执行所有与输入无关的初始化
- 构建，您知道输入张量的形状，并可以进行其余的初始化
- 调用，在哪里进行正向计算

请注意，您不必等到调用build来创建变量，您也可以在__init__中创建它们。 但是，在构建中创建它们的优点是它可以根据图层将要操作的输入的形状启用后期变量创建。 另一方面，在__init__中创建变量意味着需要明确指定创建变量所需的形状。

In [13]:
class MyDenselayer(tf.keras.layers.Layer):
    def __init__(self,num_outputs):
        super().__init__()
        self.num_outputs = num_outputs
    def build(self,input_shape):
        self.kernel = self.add_variable('kernel',shape=[input_shape[-1].value,self.num_outputs])
    def call(self,input):
        return tf.matmul(input,self.kernel)
    
layer = MyDenselayer(10)
print(layer(tf.zeros([10,5])))
print(layer.variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_denselayer_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[-0.596848  ,  0.32385546,  0.06267101, -0.0054456 , -0.6222967 ,
         0.10687149,  0.45853   ,  0.4054851 , -0.00810581,  0.05108315],
       [ 0.43627125, -0.07360744, -0.17908624, -0.05440187,  0.37792736,
        -0.08139873, -0.16967502,  0.05431771, -0.13397783,  0.5719417 ],
       [ 0.08511943,  0.61036724, -0.42390466, -0.32158184,  0.18217063,
         0.54654795,  0.19325376, -0.14536798, -0.2848401 , -0.57637024],
       [-0.38627356,  0.15984356, -0.4421542 ,  0.5119062 ,  0.41315037,
        -0.14874807,  0.30220544,  

## 模型：组成图层
机器学习模型中许多有趣的层状事物是通过组合现有层来实现的。 例如，resnet中的每个残余块是卷积，批量标准化和快捷方式的组合。

创建包含其他图层的类似图层的东西时使用的主类是tf.keras.Model。 实现一个是通过继承自tf.keras.Model完成的。

In [17]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self,kernel_size,fliters):
        super().__init__(name = '')
        filters1,filters2,filters3 = fliters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1,(1,1))
        self.bn2a = tf.keras.layers.BatchNormalization()
        
        self.conv2b = tf.keras.layers.Conv2D(filters2,kernel_size,padding='SAME')
        self.bn2b = tf.keras.layers.BatchNormalization()
        
        self.conv2c = tf.keras.layers.Conv2D(filters3,(1,1))
        self.bn2c = tf.keras.layers.BatchNormalization()
    def call(self,input_tensor,training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x,training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2b(x)
        x = self.bn2b(x,training=training)
        x= tf.nn.relu(x)
        
        x = self.conv2c(x)
        x = self.bn2c(x,training = training)
        
        x += input_tensor
        return tf.nn.relu(x)

block = ResnetIdentityBlock(1,[1,2,3])
print(block(tf.zeros([1,2,3,3])))
print([x.name for x in block.variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block_2/conv2d_9/kernel:0', 'resnet_identity_block_2/conv2d_9/bias:0', 'resnet_identity_block_2/batch_normalization_9/gamma:0', 'resnet_identity_block_2/batch_normalization_9/beta:0', 'resnet_identity_block_2/conv2d_10/kernel:0', 'resnet_identity_block_2/conv2d_10/bias:0', 'resnet_identity_block_2/batch_normalization_10/gamma:0', 'resnet_identity_block_2/batch_normalization_10/beta:0', 'resnet_identity_block_2/conv2d_11/kernel:0', 'resnet_identity_block_2/conv2d_11/bias:0', 'resnet_identity_block_2/batch_normalization_11/gamma:0', 'resnet_identity_block_2/batch_normalization_11/beta:0', 'resnet_identity_block_2/batch_normalization_9/moving_mean:0', 'resnet_identity_block_2/batch_normalization_9/moving_variance:0', 'resnet_identity_block_2/batch_normalization_10/moving_mean:0', 'resnet_identity_block_2/batch_normalization_10/moving_var

In [15]:
my_seq = tf.keras.Sequential([
    tf.keras.layers.Conv2D(1,(1,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(2,1,padding='SAME'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3,(1,1)),
    tf.keras.layers.BatchNormalization()
])

my_seq(tf.zeros([1,2,3,3]))

<tf.Tensor: id=539, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>